
<h1><center> Minicurso: Topic Modeling using Transformer</center>

<center>
<img src="https://faroldenoticias.com.br/wp-content/uploads/2023/11/jati.png" align="center">
</center>

<h2>Ministrante: Michael Oliveira da Cruz</h2>

<h3> Referências: </h3>

<a ref="https://medium.com/@nick-tan/topic-modeling-with-bertopic-a-cookbook-with-an-end-to-end-example-part-1-3ef739b8d9f8">Acesse aqui sobre: BERTopic</a>

<a ref="https://www.analyticsvidhya.com/blog/2021/06/part-2-topic-modeling-and-latent-dirichlet-allocation-lda-using-gensim-and-sklearn/">Acess aqui sobre: LDA </a>

<a ref="https://medium.com/@samia.khalid/bert-explained-a-complete-guide-with-theory-and-tutorial-3ac9ebc8fa7c">Acess aqui sobre: BERT </a>





### Algumas libraries para serem instaladas

In [ ]:
! python -m spacy download en_core_web_lg
! pip install unidecode
! pip install pyLDAvis

2023-12-08 13:55:00.581534: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-08 13:55:00.581583: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-08 13:55:00.581617: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-08 13:55:00.589407: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-08 13:55:01.651501: W tensorflow/compiler/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 56.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
cupy-cuda11x 11.0.0 requires numpy<1.26,>=1.20, but you have numpy 1.26.2 which is incompatible.


### Bibliotecas

In [ ]:
import os
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from unidecode import unidecode

import gensim
import pyLDAvis.gensim
import pickle
import pyLDAvis

import spacy
nlp = spacy.load('en_core_web_lg')

nltk.download('punkt')
nltk.download('stopwords')

/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/dtypes.py:35: DeprecationWarning: ml_dtypes.float8_e4m3b11 is deprecated. Use ml_dtypes.float8_e4m3b11fnuz
  from tensorflow.tsl.python.lib.core import pywrap_ml_dtypes
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Base de Dados

Aqui temos uma base de reviews/avaliação de um restaurantes:

* A coluna Review é o comentário dos usuários
* A coluna Liked informa se o usuário gostou ou não

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/JATI/Restaurant_Reviews.tsv', sep='\t')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


### Limpeza dos dados

Aqui mostramos algumas funções para limpeza dos dados

In [ ]:
## Retirando caracteres especiais e passando texto para minúsculo
def remove_special_tokens(text):

    formated_text = re.sub(r'[^a-zA-Z\s]+', ' ', text)

    return formated_text.lower()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
## retirada de espaços em brancos
def remove_space(text):
  # clean_text = re.sub('\s+', ' ', text)
  clean_text = re.sub('\s{2,}', ' ', text)
  return clean_text

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:4: DeprecationWarning: invalid escape sequence '\s'
<>:4: DeprecationWarning: invalid escape sequence '\s'
<ipython-input-10-8f30bd3c20f1>:4: DeprecationWarning: invalid escape sequence '\s'
  clean_text = re.sub('\s{2,}', ' ', text)


In [ ]:
## Retirando números
def remove_numbers(text):
  clean_text = re.sub('r[0-9]','', text)
  return clean_text

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
## Tokenizacao
def tokenize_text(text):

    result = ' '.join(word_tokenize(text))

    return result

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
## Lematizacao
def lematize(doc):
    doc = nlp(doc)
    sentence = []
    for token in doc:
      sentence.append(token.lemma_)

    formattedText = ""
    for word in sentence:
        formattedText = formattedText + " " +word

    return formattedText

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def remove_stop_words(example_sent):

    word_tokens = word_tokenize(example_sent)


    filtered_sentence = []

    for w in word_tokens:
        if w.lower() not in stop_words:
            filtered_sentence.append(w)

    formattedText = ""

    for word in filtered_sentence:

            formattedText = formattedText + " " +word

    formattedText = formattedText.lstrip()

    formattedText = formattedText.rstrip()

    return formattedText.lower()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
## passamos os comentários para a variável texts
raw_text = df.Review.tolist()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
raw_text[:10]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['Wow... Loved this place.',
 'Crust is not good.',
 'Not tasty and the texture was just nasty.',
 'Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.',
 'The selection on the menu was great and so were the prices.',
 'Now I am getting angry and I want my damn pho.',
 "Honeslty it didn't taste THAT fresh.)",
 'The potatoes were like rubber and you could tell they had been made up ahead of time being kept under a warmer.',
 'The fries were great too.',
 'A great touch.']

In [ ]:
from copy import copy
texts = copy(raw_text)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
## testando algumas funções
'''
Perceba que removemos os ...
'''
remove_special_tokens(texts[0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'wow  loved this place '

In [ ]:
## tokenização a nível de palavras
word_tokenize(texts[3])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['Stopped',
 'by',
 'during',
 'the',
 'late',
 'May',
 'bank',
 'holiday',
 'off',
 'Rick',
 'Steve',
 'recommendation',
 'and',
 'loved',
 'it',
 '.']

In [ ]:
## lematização
lematize(texts[3])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


' stop by during the late May bank holiday off Rick Steve recommendation and love it .'

### Juntando toda a parte de pré-processamento

In [ ]:
## Aqui juntaremos todas as partes do processamento
def pipe_clean_data(file_text):
    # print('cleaning text')
    try:

        final_text = file_text


        #removing spaces
        final_text = remove_space(final_text)
        final_text = final_text.lstrip()
        final_text = final_text.rstrip()

        clean_sentence = unidecode(final_text)
        clean_sentence = remove_special_tokens(clean_sentence)
        clean_sentence = tokenize_text(clean_sentence)
        clean_sentence = remove_stop_words(clean_sentence)


        clean_sentence = lematize(clean_sentence)

        peticao_raw_text = final_text
        peticao_clean_text = clean_sentence

        line = peticao_clean_text

        return line

    except Exception as e:
        print(f'Text to clean:{file_text}')
        print(e)
        print(type(file_text), len(file_text))
        # break


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
clean_text = []
for text in texts:
  clean_text.append(pipe_clean_data(text))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Amostragem do texto por palavras

In [ ]:
from collections import Counter
data_words = []
for item in clean_text:
    data_words+=item.split(' ')

cnt_words = Counter(data_words)
cnt_words.most_common(30)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('', 1000),
 ('food', 127),
 ('good', 121),
 ('place', 112),
 ('service', 87),
 ('go', 76),
 ('great', 71),
 ('back', 61),
 ('time', 54),
 ('like', 51),
 ('get', 42),
 ('come', 38),
 ('really', 36),
 ('well', 35),
 ('eat', 34),
 ('bad', 31),
 ('would', 29),
 ('wait', 29),
 ('love', 28),
 ('ever', 28),
 ('restaurant', 28),
 ('order', 28),
 ('also', 27),
 ('friendly', 27),
 ('nice', 27),
 ('one', 27),
 ('make', 26),
 ('say', 26),
 ('never', 26),
 ('amazing', 24)]

### Representação de dados de Texto

In [ ]:
## Aqui vamos o CountVector para criar uma matrix de Frequência de Palavras (BoW)
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words="english")
X2 = vectorizer.fit_transform(clean_text)
vectorizer.get_feature_names_out()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['absolute', 'absolutely', 'absolutley', ..., 'yum', 'yummy',
       'zero'], dtype=object)

In [ ]:
X2.toarray()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Aplicando a técnica LDA

In [ ]:
tokens_by_sentence = [word_tokenize(text) for text in clean_text]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(tokens_by_sentence)

# # # Create Corpus
texts = tokens_by_sentence

# # # Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# # View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1)]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
corpus[4]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(17, 1), (18, 1), (19, 1), (20, 1)]

In [ ]:
tokens_by_sentence[4]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['selection', 'menu', 'great', 'price']

In [ ]:
id2word[19]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'price'

In [ ]:
clean_text[4]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


' selection menu great price'

In [ ]:
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0, '0.053*"food" + 0.032*"good" + 0.021*"service" + 0.013*"great" + 0.012*"bad" + 0.010*"order" + 0.008*"time" + 0.007*"place" + 0.007*"go" + 0.007*"come"'), (1, '0.027*"great" + 0.018*"like" + 0.017*"place" + 0.015*"never" + 0.013*"service" + 0.012*"food" + 0.011*"get" + 0.011*"chicken" + 0.011*"go" + 0.009*"ever"'), (2, '0.028*"good" + 0.015*"go" + 0.013*"also" + 0.013*"place" + 0.012*"food" + 0.010*"pretty" + 0.010*"well" + 0.010*"service" + 0.008*"like" + 0.008*"really"'), (3, '0.032*"place" + 0.023*"food" + 0.015*"good" + 0.010*"steak" + 0.010*"service" + 0.009*"friendly" + 0.009*"nice" + 0.009*"well" + 0.008*"say" + 0.008*"minute"'), (4, '0.019*"food" + 0.017*"get" + 0.014*"back" + 0.012*"service" + 0.011*"well" + 0.010*"waitress" + 0.010*"really" + 0.010*"say" + 0.009*"restaurant" + 0.009*"fantastic"'), (5, '0.025*"back" + 0.024*"go" + 0.013*"good" + 0.012*"like" + 0.010*"place" + 0.010*"think" + 0.010*"service" + 0.009*"time" + 0.008*"price" + 0.008*"really"'), (6, '0.050*"go

In [ ]:
lda_model.print_topics(num_topics=10, num_words=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.053*"food" + 0.032*"good" + 0.021*"service" + 0.013*"great" + 0.012*"bad"'),
 (1,
  '0.027*"great" + 0.018*"like" + 0.017*"place" + 0.015*"never" + 0.013*"service"'),
 (2,
  '0.028*"good" + 0.015*"go" + 0.013*"also" + 0.013*"place" + 0.012*"food"'),
 (3,
  '0.032*"place" + 0.023*"food" + 0.015*"good" + 0.010*"steak" + 0.010*"service"'),
 (4,
  '0.019*"food" + 0.017*"get" + 0.014*"back" + 0.012*"service" + 0.011*"well"'),
 (5,
  '0.025*"back" + 0.024*"go" + 0.013*"good" + 0.012*"like" + 0.010*"place"'),
 (6,
  '0.050*"good" + 0.027*"place" + 0.019*"service" + 0.015*"go" + 0.013*"like"'),
 (7,
  '0.017*"place" + 0.011*"minute" + 0.010*"great" + 0.009*"come" + 0.009*"service"'),
 (8,
  '0.020*"wait" + 0.019*"great" + 0.015*"come" + 0.015*"food" + 0.014*"service"'),
 (9,
  '0.020*"food" + 0.019*"time" + 0.016*"eat" + 0.009*"great" + 0.009*"place"')]

In [ ]:
# ## Iremos instalar essa versão porque há conflito do pandas pré-instalado com a library pyLDAvis
# ! pip install pandas==1.5.1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.036315 -0.009492       1        1  12.455723
1      0.068497  0.085799       2        1  11.925272
3     -0.056953 -0.063669       3        1  10.703869
9     -0.043779  0.027302       4        1  10.617628
0     -0.014994  0.072691       5        1  10.501160
5      0.005799  0.064271       6        1   9.707094
8      0.044740  0.004997       7        1   9.576564
2     -0.044804 -0.021194       8        1   8.630701
7      0.140606 -0.094395       9        1   7.986749
4     -0.062797 -0.066309      10        1   7.895240, topic_info=        Term       Freq      Total Category  logprob  loglift
4       good  94.000000  94.000000  Default  30.0000  30.0000
82      food  98.000000  98.000000  Default  29.0000  29.0000
316     wait  23.000000  23.000000  Default  28.0000  28.0000
17     great  56.000000  56.000000  Default  27.0000  27.0000
55   chicken  15.000000  15.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
217      one   2.302005  21.618072  Topic10  -5.2374   0.2992
44        go   2.301678  59.941169  Topic10  -5.2376  -0.7208
297     dish   1.880912  13.867702  Topic10  -5.4395   0.5411
4       good   2.213526  94.642985  Topic10  -5.2766  -1.2166
33      make   1.901758  20.939948  Topic10  -5.4284   0.1400

[748 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
223       2  0.143488  absolutely
223       5  0.143488  absolutely
223       6  0.286975  absolutely
223       7  0.143488  absolutely
223      10  0.286975  absolutely
...     ...       ...         ...
687       8  0.649730        yeah
846      10  0.661021       yucky
697      10  0.886772         yum
546       3  0.440010        zero
546       4  0.440010        zero

[1475 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 2, 4, 10, 1, 6, 9, 3, 8, 5])

In [ ]:
count = 0
for i in lda_model[corpus]:
  print(f'Doc:{count}:{i}')
  count+=1
  if count ==10:
    break

Doc:0:[(0, 0.025002327), (1, 0.025004083), (2, 0.025003372), (3, 0.025006743), (4, 0.025002582), (5, 0.025005374), (6, 0.02500596), (7, 0.7749612), (8, 0.025005773), (9, 0.025002591)]
Doc:1:[(0, 0.0333383), (1, 0.033334866), (2, 0.033337783), (3, 0.033336196), (4, 0.03333499), (5, 0.033335924), (6, 0.69997686), (7, 0.0333346), (8, 0.03333511), (9, 0.033335365)]
Doc:2:[(0, 0.025006564), (1, 0.025017718), (2, 0.025003392), (3, 0.02500327), (4, 0.025002826), (5, 0.025003342), (6, 0.7749538), (7, 0.025003422), (8, 0.025002826), (9, 0.025002826)]
Doc:3:[(0, 0.010002501), (1, 0.010008467), (2, 0.010002501), (3, 0.010005552), (4, 0.010002938), (5, 0.010004888), (6, 0.010004427), (7, 0.9099614), (8, 0.010003915), (9, 0.010003445)]
Doc:4:[(0, 0.020005422), (1, 0.02000667), (2, 0.020001912), (3, 0.020002153), (4, 0.02000142), (5, 0.020003587), (6, 0.020003056), (7, 0.020002251), (8, 0.020005519), (9, 0.81996804)]
Doc:5:[(0, 0.01667), (1, 0.01667368), (2, 0.016669398), (3, 0.016673595), (4, 0.849

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### BERTopic

In [ ]:
! pip install bertopic

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 30.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 8.1 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039167 sha256=ebf1ccec7371fadeea37521f64970100a247daf2fe357c12b8fbb14b80b929da
  Stored in di

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
model_embedding = SentenceTransformer('all-MiniLM-L6-v2')
corpus_embeddings = model_embedding.encode(raw_text)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
model = BERTopic(
    n_gram_range=(1, 2),
    vectorizer_model=vectorizer,
    nr_topics='auto',
    min_topic_size=10,
    seed_topic_list=[
        ["experience", "bad", "good", "nice"],
        ["place", "atmosphere", "toilet", "clean"],
        ["staff", "waitress", "service"],
        ["wait", "time", "long"],
        ["food", "taste"]
    ],
    calculate_probabilities=True).fit(raw_text, corpus_embeddings)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
topics, probabilities = model.transform(raw_text, corpus_embeddings)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
topics[:5], probabilities[:5]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



([2, -1, 0, -1, 6],
 array([[2.09140504e-07, 5.88390602e-08, 8.83526939e-01, 7.82791562e-08,
         8.22995719e-08, 4.63783018e-08, 8.53242129e-08, 5.05066781e-08,
         8.07877367e-08, 1.72362382e-07, 5.57374068e-08, 9.55164153e-08,
         7.09435656e-08, 5.48239500e-08, 6.64440894e-08, 4.14184987e-08,
         8.85445944e-08, 6.76905606e-08, 1.67089819e-07],
        [8.16222323e-02, 4.28736357e-44, 1.57869216e-02, 6.52280406e-36,
         8.23149613e-24, 2.12886200e-44, 1.90294202e-02, 2.19357318e-44,
         1.84777007e-02, 4.83696919e-02, 1.28010702e-45, 4.46291041e-28,
         2.22651581e-02, 2.03423234e-44, 4.42428995e-02, 1.07917988e-45,
         2.31788757e-02, 3.80145764e-02, 1.73512169e-02],
        [9.99999997e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00

In [ ]:
model.get_topic_freq()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic,Count
1,-1,349
2,0,119
4,1,62
0,2,61
5,3,61
8,4,48
10,5,45
3,6,38
6,7,34
7,8,24


In [ ]:
model.visualize_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
model.visualize_barchart(top_n_topics=20)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
from umap import UMAP

reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(corpus_embeddings)
model.visualize_documents(raw_text, reduced_embeddings=reduced_embeddings)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
topics_per_class = model.topics_per_class(raw_text, classes=df.Liked.to_list())
model.visualize_topics_per_class(topics_per_class, top_n_topics=19)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

